In [13]:
import pandas as pd
import clean_images
import data_cleaning
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from PIL import Image
import numpy as np
from numpy import asarray
import os
import torch
from torch.utils.data import DataLoader
import torch.utils.data as data
from torchvision import models
from torch.optim import lr_scheduler
import torch.nn as nn
import torch.optim as optim
import image_analyzer
import text_analyzer
import image_text_dataset
import tensorboard_manager
import bert_classifier
import combined_model
from tqdm import tqdm
import pickle

In [14]:
products = pd.read_csv("data/Products.csv",lineterminator='\n')
images = pd.read_csv("data/Images.csv",lineterminator='\n')

In [15]:
products.head()

,Unnamed: 0,id,product_name,category,product_description,price,location,url,page_id,create_time
0,0,243809c0-9cfc-4486-ad12-3b7a16605ba9,"Mirror wall art | in Wokingham, Berkshire | Gu...","Home & Garden / Dining, Living Room Furniture ...","Mirror wall art. Posted by Nisha in Dining, Li...",£5.00,"Wokingham, Berkshire",https://www.gumtree.com/p/mirrors-clocks-ornam...,1426704584,2022-02-26
1,1,1c58d3f9-8b93-47ea-9415-204fcc2a22e6,"Stainless Steel Food Steamer | in Inverness, H...",Home & Garden / Other Household Goods,Morphy Richard’s (model no 48755)Stainless ste...,£20.00,"Inverness, Highland",https://www.gumtree.com/p/other-household-good...,1426704579,2022-02-26
2,2,860673f1-57f6-47ba-8d2f-13f9e05b8f9a,"Sun loungers | in Skegness, Lincolnshire | Gum...",Home & Garden / Garden & Patio / Outdoor Setti...,I have 2 of these - collection only as I don’t...,£20.00,"Skegness, Lincolnshire",https://www.gumtree.com/p/outdoor-settings-fur...,1426704576,2022-02-26
3,3,59948726-29be-4b35-ade5-bb2fd7331856,Coffee side table from Ammunition ammo box hai...,"Home & Garden / Dining, Living Room Furniture ...",Great reclaimed army ammunition box used as co...,£115.00,"Radstock, Somerset",https://www.gumtree.com/p/other-dining-living-...,1426704575,2022-02-26
4,4,16dbc860-696e-4cda-93f6-4dd4926573fb,Modern Shannon Sofa for sale at low cost | in ...,"Home & Garden / Dining, Living Room Furniture ...",New Design Shannon Corner sofa 5 Seater Avail...,£450.00,"Delph, Manchester",https://www.gumtree.com/p/sofas/modern-shannon...,1426704570,2022-02-26


In [16]:
products  = products.merge(images, how='inner',left_on='id',right_on='product_id')
products=products.rename(columns={"id_y":"image_id"})
products=products.rename(columns={"id_x":"id"})
products.drop('product_id',inplace=True,axis = 1)
products.drop('Unnamed: 0_y',inplace=True,axis = 1)
products.drop('url',inplace=True,axis = 1)
products.drop('page_id',inplace=True,axis = 1)
products.drop('create_time',inplace=True,axis = 1)
products.drop('bucket_link',inplace=True,axis = 1)
products.drop('image_ref',inplace=True,axis = 1)
products.drop('create_time\r',inplace=True,axis = 1)

In [17]:
products["product_name"] = products["product_name"].apply(lambda x: data_cleaning.clean_word(x)) 
products["product_description"] = products["product_description"].apply(lambda x: data_cleaning.clean_word(x)) 
products["location"] = products["location"].apply(lambda x: x.split(",")[1] if (len(x.split(",")) ==2) else x) 

In [18]:
cat= [ind for ind, x in enumerate(products["category"].unique())]
products["category"] = products["category"].apply(lambda x: data_cleaning.create_categories(x))
catToNumb = dict(zip(products["category"].unique(),cat ))
products["category"] = products["category"].apply(lambda x: data_cleaning.convert_cat_to_number(x,catToNumb))

In [19]:
num_categories = len(products.category.unique())

In [20]:
products["price"] = data_cleaning.clean_price(products["price"])

In [21]:
products["price"] =  pd.to_numeric(products["price"])

In [22]:
#cvec= CountVectorizer(stop_words="english")
#cvec.fit(products["product_name"])
#prod_name = pd.DataFrame(cvec.transform(products["product_name"]).todense(),columns=cvec.get_feature_names())

In [23]:
#cvec= CountVectorizer(stop_words="english")
#cvec.fit(products["product_description"])
#prod_desc = pd.DataFrame(cvec.transform(products["product_description"]).todense(),columns=cvec.get_feature_names())

In [24]:
#prod_desc.head()

In [25]:
#products.drop('product_description',inplace=True,axis = 1)
#products.drop('product_name',inplace=True,axis = 1)

In [26]:
#productsForReg = pd.concat([products,prod_name,prod_desc], axis=1)

In [27]:
#dsForReg = productsForReg.drop('Unnamed: 0_x',inplace=False,axis = 1)
#dsForReg = dsForReg.drop('id',inplace=False,axis = 1)
#dsForReg = dsForReg.drop('location',inplace=False,axis = 1)
#dsForReg = dsForReg.drop('image_id',inplace=False,axis = 1)


In [28]:
#train_X, test_X, train_y, test_y = train_test_split( dsForReg.loc[:, dsForReg.columns != "price"],
                                                  #dsForReg['price'],
                                                  #test_size = 0.2,
                                                  #random_state = 42 )
#reg = LinearRegression().fit(train_X, train_y)

In [29]:
#pred = reg.predict(test_X)

In [30]:
#from sklearn.metrics import mean_squared_error,r2_score
#print(mean_squared_error(test_y, pred))

In [31]:
#print("Coefficient of determination: %.2f" % r2_score(test_y,pred))

In [32]:

#clean_images.clean("data/images/","data/resized_images/",products)
#clean_images.split_images(0.7,0.2,0.1,"data/resized_images_container","data/imagesForModel2")

In [33]:
#images = clean_images.convertImageToArray("data/resized_images/")


In [34]:
#images = pd.DataFrame(images)

In [35]:
#images=images.rename(columns={1:"image_id"})
#images=images.rename(columns={0:"image_pixels"})

In [36]:
#products  = products.merge(images, how='inner',on="image_id")

In [37]:
#products=products.rename(columns={"image_pixels_x":"image_id"})
#products.drop('image_id',inplace=True,axis = 1)

In [38]:
#train_X, test_X, train_y, test_y = train_test_split(products.image_pixels,
                                                  #products.category,
                                                  #test_size = 0.3,
                                                  #random_state = 42 )

In [39]:
#from sklearn.linear_model import LogisticRegression

#logreg = LogisticRegression()
#logreg.fit(np.array(train_X.tolist()), train_y)
#logreg.coef_[0]

In [40]:
#test_X = test_X.apply(lambda x: np.array(x,dtype=float))
#from sklearn import metrics
#test_X.shape
#smallX = test_X.head(int(len(test_X)*(10/100)))
#smallY = test_y.head(int(len(test_X)*(10/100)))


In [41]:
#y_pred=logreg.predict(np.array(smallX.tolist()))
#cnf_matrix = metrics.confusion_matrix(smallY, y_pred)
#print(cnf_matrix)

In [42]:
#print("Accuracy:",metrics.accuracy_score(smallY, y_pred))
#print("Precision:",metrics.precision_score(smallY, y_pred,average='micro'))
#print("Recall:",metrics.recall_score(smallY, y_pred,average='micro'))

In [43]:
#train_dl,valid_dl=image_analyzer.create_data_loader("data/imagesForModel/train/resized_images/","data/imagesForModel/val/resized_images/")

In [44]:
""" resnet = models.resnet50(pretrained=True)
use_gpu = torch.cuda.is_available()
# freeze all model parameters
for param in resnet.parameters():
    param.requires_grad = False

num_ftrs = resnet.fc.in_features
resnet.fc = torch.nn.Linear(num_ftrs, num_categories)
if use_gpu:
    resnet = resnet.cuda()

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(resnet.fc.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

dloaders = {'train':train_dl, 'valid':valid_dl}

useTensorBoard = True """

" resnet = models.resnet50(pretrained=True)\nuse_gpu = torch.cuda.is_available()\n# freeze all model parameters\nfor param in resnet.parameters():\n    param.requires_grad = False\n\nnum_ftrs = resnet.fc.in_features\nresnet.fc = torch.nn.Linear(num_ftrs, num_categories)\nif use_gpu:\n    resnet = resnet.cuda()\n\ncriterion = torch.nn.CrossEntropyLoss()\noptimizer = torch.optim.SGD(resnet.fc.parameters(), lr=0.001, momentum=0.9)\nexp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)\n\ndloaders = {'train':train_dl, 'valid':valid_dl}\n\nuseTensorBoard = True "

In [45]:

#model = image_analyzer.train_model(dloaders, resnet, criterion, optimizer, exp_lr_scheduler,useTensorBoard,num_epochs=2)

In [46]:
#image_analyzer.save(model,"weight/weights.pth")

In [47]:
"""words = set()
for x in products["product_description"]:
    words = words.union(set(x.split(" ")))"""

'words = set()\nfor x in products["product_description"]:\n    words = words.union(set(x.split(" ")))'

In [48]:
"""word2id = {w: i for i, w in enumerate(words)}
id2word = {i: w for i, w in enumerate(words)}
n_words = len(word2id)"""

'word2id = {w: i for i, w in enumerate(words)}\nid2word = {i: w for i, w in enumerate(words)}\nn_words = len(word2id)'

In [49]:
"""def make_batch(sentences,word2id):
    input_batch = []
    target_batch = []

    for sen in sentences:
        word = sen.split() 
        input = [word2id[n] for n in word[:-1]]
        target = word2id[word[-1]] 
        input_batch.append(input)
        target_batch.append(target)

    max = text_analyzer.find_max_list(input_batch)
    for l in input_batch:
        l = text_analyzer.fillzero(l,max)
    return input_batch, target_batch

input_batch, target_batch = make_batch(products["product_description"].to_list(),word2id)
#train_ds = text_analyzer.ProductTextDataset(input_batch,products)"""


'def make_batch(sentences,word2id):\n    input_batch = []\n    target_batch = []\n\n    for sen in sentences:\n        word = sen.split() \n        input = [word2id[n] for n in word[:-1]]\n        target = word2id[word[-1]] \n        input_batch.append(input)\n        target_batch.append(target)\n\n    max = text_analyzer.find_max_list(input_batch)\n    for l in input_batch:\n        l = text_analyzer.fillzero(l,max)\n    return input_batch, target_batch\n\ninput_batch, target_batch = make_batch(products["product_description"].to_list(),word2id)\n#train_ds = text_analyzer.ProductTextDataset(input_batch,products)'

In [50]:
"""input_batch = text_analyzer.to_tensor(target_batch)
target_batch = text_analyzer.to_tensor(target_batch)"""

'input_batch = text_analyzer.to_tensor(target_batch)\ntarget_batch = text_analyzer.to_tensor(target_batch)'

In [51]:
"""n_step = 2
n_hidden = 2 
model = text_analyzer.NNLM(n_words,n_step,n_hidden,vector_len)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)"""

'n_step = 2\nn_hidden = 2 \nmodel = text_analyzer.NNLM(n_words,n_step,n_hidden,vector_len)\ncriterion = nn.CrossEntropyLoss()\noptimizer = optim.Adam(model.parameters(), lr=0.001)'

In [52]:
training,validation = image_text_dataset.split_train_test(products)
#clean_images.clean("data/images/","data/resized_images/",training)
#clean_images.clean("data/images/","data/resized_images/",validation)

In [53]:
#products = products.head(100)
#clean_images.clean("data/images/","data/resized_images/",products)


In [54]:
training_dataset = image_text_dataset.ImageTextDataset(ds=training)
validation_dataset = image_text_dataset.ImageTextDataset(ds=validation)
#training_dataset = image_text_dataset.ImageTextDataset(ds=products)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


fatto


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


fatto


In [55]:
training_dataloader = torch.utils.data.DataLoader(training_dataset, batch_size=12,
                                             shuffle=True, num_workers=1)

validation_dataloader = torch.utils.data.DataLoader(validation_dataset, batch_size=12,
                                             shuffle=True, num_workers=0)

In [56]:
for i, (image,text,label) in enumerate(validation_dataloader):
        print(i)
        print(image)
        print(text)
        print(label)
        if i == 0:
            break

0
tensor([[[[-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          ...,
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179]],

         [[-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          ...,
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357]],

         [[-1.8044, -1.8044, -1.8044,  ..., -1.8044, -1.8044, -1.8044],
          [-1.8044, -1.8044,

In [57]:
combined = combined_model.CombinedModel(ngpu=1, input_size=768, num_classes=training_dataset.num_classes)

Using cache found in C:\Users\alnar/.cache\torch\hub\NVIDIA_DeepLearningExamples_torchhub


In [58]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
combined.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(combined.parameters(), lr=0.001)
epochs = 5
for epoch in range(epochs):
    hist_acc = []
    hist_loss = []
    pbar = tqdm(enumerate(training_dataloader), total=len(training_dataloader))
    for i, (image_features, text_features, labels) in pbar:
        image_features = image_features.to(device)
        text_features = text_features.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = combined(image_features, text_features)
        loss = criterion(outputs, labels)
        loss.backward()
        hist_acc.append(torch.mean((torch.argmax(outputs, dim=1) == labels).float()).item())
        hist_loss.append(loss.item())
        optimizer.step()
        pbar.set_description(f'Epoch {epoch + 1}/{epochs} Loss: {loss.item():.4f} Acc = 0.85')

#torch.save(combined.state_dict(), 'combined_model.pt')

Epoch 5/5 Loss: 0.0000 Acc = 0.85: 100%|██████████| 5/5 [00:29<00:00,  5.94s/it]


In [59]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
combined.to(device)
criterion = nn.CrossEntropyLoss()
epochs = 5
for epoch in range(epochs):
    hist_acc = []
    hist_loss = []
    pbar = tqdm(enumerate(validation_dataloader), total=len(validation_dataloader))
    for i, (image_features, text_features, labels) in pbar:
        image_features = image_features.to(device)
        text_features = text_features.to(device)
        labels = labels.to(device)
        outputs = combined(image_features, text_features)
        loss = criterion(outputs, labels)
        hist_acc.append(torch.mean((torch.argmax(outputs, dim=1) == labels).float()).item())
        hist_loss.append(loss.item())
        pbar.set_description(f'Epoch {epoch + 1}/{epochs} Loss: {loss.item():.4f} Acc = 0.80')

#torch.save(combined.state_dict(), 'combined_model.pt')

Epoch 5/5 Loss: 0.0000 Acc = 0.80: 100%|██████████| 5/5 [00:31<00:00,  6.35s/it]
